In [ ]:
import neat
import numpy as np
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score
import itertools
from scipy.signal import savgol_filter
import matplotlib.image as mpimg
from scipy.ndimage import gaussian_filter, laplace
from scipy import ndimage
#import cv2
from scipy import interpolate
from mpl_toolkits import mplot3d
from scipy.interpolate import interpn
import well_profile as wp
import welleng as we
import plotly.graph_objects as go

In [ ]:
# well_2 = wp.get(6000, profile='J', kop=1000, eob=3000, build_angle=85, set_start={'east':2000})   
# well_2.plot(style={'darkMode': False, 'color': 'dls', 'size': 5}).show()   

In [ ]:
# hold = []
# for i in range(0,len(well_2.trajectory)):
#     if well_2.trajectory[i]['sectionType'] == 'hold':
#         hold.append(well_2.trajectory[i])

In [ ]:
from welleng.connector import Connector
from welleng.survey import from_connections
from welleng.mesh import WellMesh
from welleng.visual import plot as weplots
# import os

# os.environ['DISPLAY'] = ':1'

# define start position and vector
pos1 = [0., 0., 0.]
vec1 = [0., 0., 1.]  # this is the same as inc=0, azi=0

# for the first section, we want to hold vertical for the first 500m
md2 = 500
vec2 = [0., 0., 1.]

# let's connect those points
s1 = Connector(
    pos1=pos1,
    vec1=vec1,
    md2=md2,
    vec2=vec2
)

# next we want to build to 30 degrees towards east
inc3 = 30
azi3 = 90

# let's connect this to the end of the previous section
# use the `_target` suffix to get the last position of the previous section
s2 = Connector(
    pos1=s1.pos_target,
    vec1=s1.vec_target,
    md1=s1.md_target,
    inc2=inc3,
    azi2=azi3
)

# the subsurface target has coordinates [-800, 300, 1800]
# we want to be near horizontal when we hit this point so that we can
# drill the reservoir horizontal (let's say 88 deg) and the reseroir
# orientation is southeast-northwest and we have good directional control
pos4 = [-800., 300., 1800.]
inc4 = 82
azi4 = 100
dls_design4 = 4

s3 = Connector(
    pos1=s2.pos_target,
    vec1=s2.vec_target,
    md1=s2.md_target,
    pos2=pos4,
    inc2=inc4,
    azi2=azi4,
    dls_design=dls_design4
)

# finally, we want a 500m horizontal section in the reservoir
md5 = s3.md_target + 500
inc5 = 85
dls_design2 = 2

s4 = Connector(
    pos1=s3.pos_target,
    vec1=s3.vec_target,
    md1=s3.md_target,
    md2=md5,
    inc2=inc5,
    dls_design=dls_design2
)

# make a list of the well sections
well = [s1, s2, s3, s4]

# generate the survey listing and interpolate to get desired survey spacing
survey2 = from_connections(well, step=30)

# make a mesh
mesh2 = WellMesh(survey2, method='circle')

In [ ]:
def plot(traj_x,traj_y, traj_z):
    fig = go.Figure()

    fig.add_trace(
        go.Scatter3d(
            x=traj_x,
            y=traj_z,
            z=traj_y,
            mode='lines',
            line=dict(
                color='red',
                width = 7
            ),
            name='survey_interpolated'
        ),
    )
    fig.update_layout(scene = dict(
                  #  xaxis_title='X AXIS TITLE',
                    yaxis_title='Drilling direction',
                    zaxis_title='True vertical depth (TVD)'),
                    width=700,
                    margin=dict(r=20, b=10, l=10, t=10))

    fig.update_scenes(zaxis_autorange="reversed")
    fig.show()
    

In [ ]:
survey_corr = np.copy(survey2.get_nev_arr())
survey_corr.shape

In [ ]:
for i in range(70,102):
    survey_corr[i, 0] = survey_corr[i, 0] + np.random.uniform(20)
    survey_corr[i, 1] = survey_corr[i, 1] + np.random.uniform(10)

In [ ]:
fig = go.Figure()

fig.add_trace(
        go.Scatter3d(
            x=survey2.get_nev_arr()[:,0],
            y=survey2.get_nev_arr()[:,1],
            z=survey2.get_nev_arr()[:,2],
            mode='lines',
            line=dict(
                color='red',
                width = 2
            ),
            name='Initial'
        ),
    )

fig.add_trace(
        go.Scatter3d(
            x=survey_corr[65:,0],
            y=survey_corr[65:,1],
            z=survey_corr[65:,2],
            mode='lines',
            line=dict(
                color='blue',
                width = 7
            ),
            name='Corrected'
        ),
    )

fig.update_layout(scene = dict(
                  #  xaxis_title='X AXIS TITLE',
                    yaxis_title='Drilling direction',
                    zaxis_title='True vertical depth (TVD)'),
                    width=700,
                    margin=dict(r=20, b=10, l=10, t=10))

fig.update_scenes(zaxis_autorange="reversed")
fig.show()



#plot(survey2.get_nev_arr()[:,0], survey2.get_nev_arr()[:,2], survey2.get_nev_arr()[:,1])

# 3D DQN

In [ ]:
import itertools
import torch
from torch import nn
import gym
from gym.spaces import Discrete, Box
from gym import Env
from scipy.interpolate import interpn
import os
from geostering_env import Geosteering_env
from replay_buffer import ReplayBuffer
import sys
from tqdm import tqdm
from torch.utils.tensorboard import SummaryWriter
import pickle
import time

In [ ]:
with open('3D_301.833.2055.pickle', 'rb') as f:
         cube_3d = pickle.load(f)
volume_cut= cube_3d[:,0::3,0::4]

In [ ]:
class DeepQNetwork(nn.Module):
    def __init__(self, lr, out_dims, input_dims, name, saved_dir, hid_dim):
        super().__init__()
        self.checkpoint_file = os.path.join(saved_dir, name)
        
        self.fc1 = nn.Linear(input_dims, hid_dim)
        self.fc2 = nn.Linear(hid_dim, hid_dim)
        self.fc3 = nn.Linear(hid_dim, out_dims)
        self.relu = nn.ReLU()
        self.optimizer =  torch.optim.Adam(self.parameters(), lr=lr)
        self.loss = nn.HuberLoss()
        self.device = torch.device('cpu')
        self.to('cpu')
        
    def forward(self, state):
        x = self.relu(self.fc1(state.float()))
        x = self.relu(self.fc2(x))
        q_val = self.relu(self.fc3(x))
        
        return q_val
    
    def save_checkpoint(self):
        torch.save(self.state_dict(), self.checkpoint_file)
        

In [ ]:
class Agent:
    def __init__(self, type, gamma, lr, n_actions, n_states, 
                batch_size, mem_size, replace, saved_dir, epsilon, env_name):

        self.type = type        
        self.gamma = gamma
        self.action_space = np.arange(n_actions)
        self.batch_size = batch_size
        self.replace_num = replace
        self.epsilon = epsilon
        self.learn_idx = 0
        self.loss_plot = 0
        self.running_loss = 0
        
        self.memory = ReplayBuffer(mem_size, n_states)
        
        self.Q_eval = DeepQNetwork(lr=lr,out_dims=n_actions,input_dims=n_states,
                                   name=env_name+'.pth', saved_dir=saved_dir, 
                                   hid_dim = 128)
        self.Q_next = DeepQNetwork(lr=lr,out_dims=n_actions,input_dims=n_states,
                                   name=env_name+'_q_next.pth', saved_dir=saved_dir, 
                                   hid_dim = 128)
        
        
    def choose_action(self, observation):
        if np.random.random() > self.epsilon:
            state = torch.tensor(np.array(observation)).to(self.Q_eval.device)
            action = torch.argmax(self.Q_eval.forward(state)).item()
        else:
            action = np.random.choice(self.action_space)
            
        return action    
        
    def store_transition(self, state, action, reward, next_state, done):
        self.memory.store_transition(state, action, reward, next_state, done)

    def save_models(self):
        self.Q_eval.save_checkpoint()
    
    def learn(self):
        if self.memory.mem_idx < self.batch_size:
            return
        
        if self.learn_idx % self.replace_num == 0:
            self.Q_next.load_state_dict(self.Q_eval.state_dict())
        
        states, actions, rewards, next_states, dones = self.memory.sample_buffer(self.batch_size)
        
        batch_index = np.arange(self.batch_size, dtype=np.int32)

        self.Q_eval.optimizer.zero_grad()

        if self.type == 'dqn':
            q_eval = self.Q_eval.forward(states)[batch_index, actions]
            q_next = self.Q_next.forward(next_states).max(dim=1)[0]
            
            q_next[dones] = 0.0
            
            q_target = rewards + self.gamma * q_next
            loss = self.Q_eval.loss(q_target, q_eval).to(self.Q_eval.device)

        elif self.type == 'doubledqn':
            q_pred = self.Q_eval.forward(states)[batch_index, actions]
            q_next = self.Q_next.forward(next_states)
            q_eval = self.Q_eval.forward(next_states)
            
            max_actions = T.argmax(q_eval, dim=1)
            q_next[dones] = 0.0
            
            q_target = rewards + self.gamma * q_next[batch_index, max_actions]

            loss = self.Q_eval.loss(q_target, q_pred).to(self.Q_eval.device)

        loss.backward()
        self.Q_eval.optimizer.step()

        self.running_loss += loss.item()
        self.learn_idx += 1

In [ ]:
agent = Agent(type = 'doubledqn', gamma = 0.99, lr = 4e-3, n_actions = 9, n_states = 3, 
                batch_size = 64, mem_size = 25000, replace = 1000, saved_dir = './', epsilon = 0.4, env_name = 'Geosteering_env')

In [ ]:
best_score = -np.inf
env = Geosteering_env(action_space = 9, prod_map = volume_cut, start_point = [140, 60, 20], init_azimut = 0,
                      init_inclination = 0, length = 10, step_incl = 1, step_azimuth = 1, angle_constraint_per_m = 1)

In [ ]:
eps_min = 0.01
n_episodes = 100
eps_dec = 0.99995
scores = np.zeros([n_episodes])
best_score = -np.inf
avg_score = np.zeros([n_episodes])
for i in tqdm(range(n_episodes)):
            done = False
            obs = env.reset()
            score = 0
            while not done:
                action = agent.choose_action(obs)
                obs_, reward, done, info = env.step(action)
                
                score += reward
                agent.store_transition(observation, action,
                                           reward, observation_, done)
                agent.learn()

                if agent.learn_idx%500:
                    loss_plot = agent.running_loss/500
                    agent.running_loss = 0
                    writer.add_scalar("loss", loss_plot, bd_agent.learn_idx)
                obs = obs_
    
            agent.epsilon = agent.epsilon*eps_dec \
            if agent.epsilon > eps_min else eps_min

            scores[i] = score
            if i>= 100:
                avg_score[i] = np.mean(scores[i-99:i+1])
                
            if avg_score[i] > best_score and i>= 100:
                agent.save_models()
                best_score = avg_score[i]
                      
            if writer_stat:
                writer.add_scalar("reward_100", avg_score[i], i)
                writer.add_scalar("reward", scores[i], i)
                

end = time.time()
elapsed_time = (end-start)
start = end
print('======== Finished with elapsed time of %.2f' %elapsed_time, 'seconds ========')
if writer_stat:
    writer.close()